In [1]:
%pip install duckdb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
DB_PATH = "../formatted_zone/formatted.db"

In [3]:
from os import listdir
from os.path import isfile, join, isdir

def getAllFilesRecursive(root):
    root = os.path.normpath(root)
    
    files = [join(root, f) for f in os.listdir(root) if isfile(join(root, f))]
    dirs = [d for d in os.listdir(root) if isdir(join(root, d))]

    for d in dirs:
        files_in_d = getAllFilesRecursive(join(root, d))
        if files_in_d:
            files.extend(files_in_d)

    return files

In [47]:
import duckdb
import os
file = '../persistent_landing/fotocasa/2024/2024_09_27_fotocasa.json'
with duckdb.connect(DB_PATH) as con:
    con.sql(f"CREATE TABLE IF NOT EXISTS {os.path.basename(file).split('.')[0]} AS FROM read_json('{file}');")

json


ParserException: Parser Error: syntax error at or near "2024_09_27"

In [4]:
import duckdb
import os
SOURCE = '../persistent_landing/'

files = getAllFilesRecursive(SOURCE)
with duckdb.connect(DB_PATH) as con:
    con.sql("""
        INSTALL spatial;
        LOAD spatial;
        """)
    for file in files:
        file = file.replace("\\", "/")
        filename = os.path.basename(file).split('.')
        print(file)
        read_function = f'read_{filename[-1]}'
        if filename[-1] == 'xlsx':
            read_function = 'st_read'
        con.sql(f"CREATE TABLE IF NOT EXISTS {'_'.join(filename[0].split('_')[::-1])} AS FROM {read_function}('{file}');")
        if filename[0].split('_')[0].isdigit():
            con.sql(f"ALTER TABLE {'_'.join(filename[0].split('_')[::-1])} ADD COLUMN timestamp VARCHAR DEFAULT '{'-'.join(filename[0].split('_')[:-1])}';")
            print('-'.join(filename[0].split('_')[:-1]))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

../persistent_landing/fotocasa/2024/2024_09_27_fotocasa.json
2024-09-27
../persistent_landing/fotocasa/2024/2024_09_28_fotocasa.json
2024-09-28
../persistent_landing/idealista/2020/2020_01_02_idealista.json
2020-01-02
../persistent_landing/idealista/2020/2020_01_08_idealista.json
2020-01-08
../persistent_landing/idealista/2020/2020_01_10_idealista.json
2020-01-10
../persistent_landing/idealista/2020/2020_01_13_idealista.json
2020-01-13
../persistent_landing/idealista/2020/2020_01_23_idealista.json
2020-01-23
../persistent_landing/idealista/2020/2020_01_24_idealista.json
2020-01-24
../persistent_landing/idealista/2020/2020_01_28_idealista.json
2020-01-28
../persistent_landing/idealista/2020/2020_01_30_idealista.json
2020-01-30
../persistent_landing/idealista/2020/2020_02_07_idealista.json
2020-02-07
../persistent_landing/idealista/2020/2020_02_10_idealista.json
2020-02-10
../persistent_landing/idealista/2020/2020_02_11_idealista.json
2020-02-11
../persistent_landing/idealista/2020/2020_

In [5]:
# Example to show the tables can be joined
with duckdb.connect(DB_PATH) as con:
    response = con.sql("SELECT * FROM income_2021")
    print(response)

┌──────────────┬──────────────────────────────────────────────┬───────────┬───────────────┬───────────┬───────────────┬───────────┐
│   Distric    │                    Barris                    │ RDLpc (€) │ Index (RDLpc) │ RPLpc (€) │ Index (RPLpc) │ timestamp │
│   varchar    │                   varchar                    │  double   │    varchar    │  double   │    varchar    │  varchar  │
├──────────────┼──────────────────────────────────────────────┼───────────┼───────────────┼───────────┼───────────────┼───────────┤
│ Ciutat Vella │ el Raval                                     │    11.178 │ 0.54          │    11.919 │ 0.51          │ 2021      │
│ Ciutat Vella │ el Barri Gòtic                               │     15.99 │ 0.77          │    18.668 │ 0.80          │ 2021      │
│ Ciutat Vella │ la Barceloneta                               │    14.943 │ 0.72          │    15.659 │ 0.67          │ 2021      │
│ Ciutat Vella │ Sant Pere, Santa Caterina i la Ribera        │     16.77 │ 